In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np

from my_stuff import MufexKeys
import requests
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.nb_funcs.nb_strategies.nb_strategy import (
    nb_strat_bt_create_ind,
    nb_strat_evaluate,
    nb_strat_get_current_ind_settings,
    nb_strat_get_ind_set_str,
    nb_strat_get_total_ind_settings,
)
from quantfreedom.nb_funcs.nb_simulate import nb_sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
from quantfreedom.nb_funcs.nb_sim_order_records import nb_sim_get_or


mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles_np = mufex_main.get_candles(symbol="BTCUSDT", timeframe="5m", candles_to_dl=1000)

It took 00 mins and 01 seconds to download 1000 candles


In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=10,
    gains_pct_filter=-90,
    total_trade_filter=60,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([6, 9, 12]),
    max_trades=np.array([3]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 6]),
    sl_based_on_add_pct=np.array([0.1, 0.5, 1., .05]),
    sl_based_on_lookback=np.array([10, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([.5, 1, 2]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [5]:
nb_sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 432
Total combinations of settings to test: 2,592
Total candles: 1,000
Total candles to test: 2,592,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,36,61.0,14,47,-65.28,22.95,0.0,622.533,-652.8264,347.1736
1,0,96,61.0,13,48,-75.41,21.31,0.0,633.255,-754.0713,245.9287
2,0,132,75.0,12,63,-82.11,16.00,0.0,764.849,-821.0660,178.9340
3,0,180,68.0,12,56,-82.66,17.65,0.0,754.355,-826.6408,173.3592
4,0,228,71.0,10,61,-84.88,14.08,0.0,863.196,-848.8023,151.1977
5,0,229,63.0,9,54,-83.36,14.29,0.0,740.708,-833.5853,166.4147
6,0,276,76.0,11,65,-89.02,14.47,0.0,836.755,-890.1974,109.8026
7,0,277,64.0,9,55,-85.24,14.06,0.0,730.485,-852.4306,147.5694
8,1,36,61.0,13,48,-66.37,21.31,0.0,608.821,-663.7176,336.2824
9,1,96,61.0,13,48,-75.41,21.31,0.0,633.255,-754.0713,245.9287


In [6]:
order_records_df, data = nb_sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=nb_strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=nb_strat_get_current_ind_settings,
    get_ind_set_str=nb_strat_get_ind_set_str,
    get_total_ind_settings=nb_strat_get_total_ind_settings,
    ind_creator=nb_strat_bt_create_ind,
    nb_logger_type=NBLoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=2,
    dos_index=36,
)

In [7]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,2,36,100,1698153600000,2023-10-24 13:20:00,EntryFilled,1000.0000,932.7980,4547.833,67.202,...,4615.035,34412.2,NaN,0.134,4615.035,NaN,0.47,34250.3,1.4,34901.0
1,2,36,101,1698153900000,2023-10-24 13:25:00,EntryFilled,1000.0000,873.8010,8380.894,126.199,...,3959.260,34449.5,NaN,0.249,8574.295,NaN,0.50,34250.3,1.5,34947.2
2,2,36,106,1698155400000,2023-10-24 13:50:00,StopLossFilled,941.3588,941.3588,NaN,NaN,...,NaN,NaN,34250.3,NaN,NaN,-58.6412,NaN,NaN,NaN,NaN
3,2,36,107,1698155700000,2023-10-24 13:55:00,EntryFilled,941.3588,884.8680,3480.471,56.491,...,3536.962,34335.7,NaN,0.103,3536.962,NaN,0.61,34125.5,1.7,34920.7
4,2,36,108,1698156000000,2023-10-24 14:00:00,EntryFilled,941.3588,841.6940,5941.367,99.665,...,2560.561,34440.4,NaN,0.177,6097.523,NaN,0.70,34125.5,2.0,35039.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,2,36,979,1698417300000,2023-10-27 14:35:00,EntryFilled,180.1763,165.6040,1171.556,14.572,...,1186.128,33918.3,NaN,0.035,1186.128,NaN,0.24,33836.3,1.0,34245.1
259,2,36,980,1698417600000,2023-10-27 14:40:00,EntryFilled,180.1763,152.7530,2173.254,27.423,...,1029.121,33940.2,NaN,0.065,2215.249,NaN,0.26,33836.3,1.0,34272.3
260,2,36,985,1698419100000,2023-10-27 15:05:00,StopLossFilled,170.4338,170.4338,NaN,NaN,...,NaN,NaN,33836.3,NaN,NaN,-9.7425,NaN,NaN,NaN,NaN
261,2,36,986,1698419400000,2023-10-27 15:10:00,EntryFilled,170.4338,157.3220,992.484,13.112,...,1005.596,33920.9,NaN,0.030,1005.596,NaN,0.32,33813.2,1.1,34299.0


In [8]:
res = requests.post("http://127.0.0.1:7779/plot", json=data)

# Convert response data to json
returned_data = res.json()

print(returned_data)

{'ok': True}
